In [1]:
#import packages

import numpy as np
import seaborn as sns 
from geopy.distance import distance
import scipy.stats as stats
from scipy import stats 
from math import sqrt
import scipy
import pandas as pd
import sklearn 
from sklearn.preprocessing import StandardScaler
from statsmodels.formula.api import ols 
import matplotlib.pyplot as plt   
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn import linear_model 
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler 
import statsmodels.formula.api as smf
import statsmodels.api as sm 
import pickle 



plt.style.use('seaborn')
sns.set(style='darkgrid', color_codes=True)

In [2]:
scaler_file = open('scaler.pickle','rb')
final_scaler = pickle.load(scaler_file)
model_file = open('model.pickle','rb')
final_model = pickle.load(model_file)
scaler_file.close()
model_file.close()

In [3]:
#Read in hold data and niche file

h = pd.read_csv('kc_house_data_test_features.csv')  #insert prediction file
niche = pd.read_csv('Niche - Sheet1.csv') 

In [4]:
#joins are original data set with data from niche.com that includes population data, school grades, and nieghborhood grades
hnj = niche.set_index('zipcode').join(h.set_index('zipcode')) 
hnj = hnj.reset_index()
hnj = hnj.dropna(subset=['id'])

In [5]:
#Creates variable list tranform thruout
prelim_feat = ['zipcode', 'niche_grade', 'school_grade', 'population', 'date', 'price', 'bedrooms', 'bathrooms',
       'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition','grade', 'sqft_above', 
               'sqft_basement', 'yr_built', 'yr_renovated','lat', 'long'] 

test_feat = ['zipcode', 'population', 'bedrooms', 'bathrooms',
       'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition','grade', 'sqft_above', 
               'sqft_basement', 'yr_built', 'yr_renovated','lat', 'long'] 


hnj1 = hnj[test_feat] 
target = 'price'

In [6]:
#Drops unneeded columns
def drop_uneeded_columns(df): 
    unwanted_columns_lists= ['zip_rank','Unnamed: 0', 'id', 'date',
                             'sqft_living15', 'sqft_lot15'] 
    for c in unwanted_columns_lists: 
        del df[c] 

In [7]:
#call function above
drop_uneeded_columns(hnj)

In [8]:
#This function caps bedrooms and bathrooms 
def cap_ba_bd(row):
    if row['bedrooms'] > 11:
        row['bedrooms'] = 3 #column average
    if row['bedrooms'] > 9:
        row['bedrooms'] = 10
    if row['bathrooms'] < 1:
        row['bathrooms'] = 1
    return row  

hnj = hnj.apply(cap_ba_bd, axis = 1) 

In [9]:
clean_data = hnj.copy()

In [10]:
#Calculates the distance to a popular park in seattle and creates the column 
clean_data['distance_to_gas_works'] = np.nan
def dist_to_gas_work(row): 
    gas_works_park = (47.6456, -122.3344) 
    coord = (row['lat'], row['long']) 
    dist = distance(gas_works_park, coord).miles 
    row['distance_to_gas_works'] = (round((dist), 2))
    return row 

clean_data = clean_data.apply(dist_to_gas_work, axis = 1)

In [11]:
#Calculates the ratio of bedrooms to bathrooms
clean_data['bath_to_bed'] = np.nan
def b_b_column(row): 
    if row['bedrooms'] == 0: 
         bath_bed = row['bathrooms']/1
    else:
        bath_bed = row['bathrooms']/row['bedrooms'] 
    ratio_ab = abs(bath_bed) 
    row['bath_to_bed'] = ratio_ab 
    return row 

clean_data = clean_data.apply(b_b_column, axis = 1)

In [12]:
#Creates a binary column that denotes whether a home has the golden ratio (or is within 10% of it)
clean_data['golden_ratio'] = np.nan
def golden_ratio(row): 
    golden_ratio = 2/3
    golden_ratio_u = golden_ratio + (golden_ratio*.10) 
    golden_ratio_l = golden_ratio - (golden_ratio*.10) 
    if row['bath_to_bed'] >= golden_ratio_l:
        if row['bath_to_bed'] <= golden_ratio_u: 
            row['golden_ratio'] = 1  
        else: 
            row['golden_ratio'] = 0 
    else: 
        row['golden_ratio'] = 0 
    return row 

clean_data = clean_data.apply(golden_ratio, axis = 1)

In [13]:
#Creates a populates a sqft living to lot ration 
clean_data['sqft_li_to_sqft_lo'] = np.nan
def li_lo_column(row): 
    living__to_lot = row['sqft_living']/row['sqft_lot']
    ratio_ab = abs(living__to_lot) 
    row['sqft_li_to_sqft_lo'] = ratio_ab 
    return row  
clean_data = clean_data.apply(li_lo_column, axis = 1)

In [14]:
#Creates and populates a new row tha combine grade and condition while slightly weighing grade by 25%
clean_data['score'] = np.nan
def grade_n_cond(row): 
    new_grade = row['grade']*1.25
    score = new_grade + row['condition']
    row['score'] = score 
    return row 

clean_data = clean_data.apply(grade_n_cond, axis = 1)

In [15]:
#Creates and populates a new column that shows when the house was last changed whether it was when it was built or renovated
clean_data['year_updated'] = np.nan
def year_changed(row): 
    if row['yr_renovated'] > 1:
        row['year_updated'] = row['yr_renovated']  
    else: 
        row['year_updated'] = row['yr_built']
    return row 

clean_data = clean_data.apply(year_changed, axis = 1)

In [16]:
#Creates a binary based on whether or not a home has a basement by noting if it includes a number in higher zero in the sqft_basement column
clean_data['basement'] = clean_data['sqft_basement'].map(lambda x : 1 if x != 0 else 0) 

#Changes niche_grades from letters to numbers 
clean_data['niche_grade'] = clean_data['niche_grade'].replace(['A+','A+ ','A','A-','B+','B','B-'],[6,6,5,4,3,2,1])


#Changes school grades from letters to numbers
clean_data['school_grade'] = clean_data['school_grade'].replace(['A+','A+ ','A ','A','A-','A- ','B+','B','B-','C+'],[7,7,6,6,5,5,4,3,2,1])

In [17]:
#Catagorizes homes by number of bedrooms 
clean_data['home_category'] = np.nan
def home_cat(row): 
    if row['bedrooms'] < 2:
        row['home_category'] = 'starter'  
    elif row['bedrooms'] < 5: 
        row['home_category'] = 'small_family' 
    elif row['bedrooms'] < 9: 
        row['home_category'] = 'large_family'
    else:
        row['bedrooms'] >= 9
        row['home_category'] = 'mansion'
    return row 

clean_data = clean_data.apply(home_cat, axis = 1)

In [18]:
#Catagorizes home based on score 
clean_data['score_cat'] = np.nan
def score_cat(row): 
    if row['score'] <= 11.5:
        row['score_cat'] = 'low'  
    elif row['score'] <= 12: 
        row['score_cat'] = 'low_mid' 
    elif row['score'] <= 13: 
        row['score_cat'] = 'high_mid'
    else:
        row['score_cat'] = 'high_end'
    return row 

clean_data = clean_data.apply(score_cat, axis = 1)

In [19]:
#Calculates the last year updated from either year built or year renovated and populates it
clean_data['years_since_updated'] = np.nan
def years_old(row):  
    age = 2021 - row['year_updated'] 
    if age < 1: 
        row['years_since_updated'] = 1 
    else: 
        row['years_since_updated'] = age
    return row 
clean_data = clean_data.apply(years_old, axis = 1)

In [20]:
clean_data2 = clean_data.copy()

In [21]:
#drop columns that are no longer needed because new features have been created
def drop_uneeded_columns2(df): 
    unwanted_columns_lists= ['grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated',
        'long', 'lat', 'condition'] 
    for c in unwanted_columns_lists: 
        del df[c]  

drop_uneeded_columns2(clean_data2)  

In [22]:
pf_data = clean_data2.copy()

In [23]:
#Turns out home category into dummy variables and drops the original column
pf_data = pd.concat([pf_data, pd.get_dummies(pf_data['home_category'])], 1) 
pf_data = pf_data.drop(columns = 'home_category') 

In [24]:
#Turns our score categories into dummy variables and drops the original column
pf_data = pd.concat([pf_data, pd.get_dummies(pf_data['score_cat'])], 1)   
pf_data = pf_data.drop(columns = 'score_cat') 

In [25]:
#Turns zipcodes into dummy variables and drops original columns
pf_data = pd.concat([pf_data, pd.get_dummies(pf_data['zipcode'])], 1)  
pf_data = pf_data.drop(columns = 'zipcode') 

In [26]:
pf_data2 = pf_data.copy()

In [27]:
pf_data2['niche_grade^2'] = pf_data2['niche_grade'] * pf_data2['niche_grade'] 
pf_data2['niche_grade_sqft_li_to_sqft_lo'] = pf_data2['sqft_li_to_sqft_lo'] * pf_data2['niche_grade'] 
pf_data2['niche_grade_score'] = pf_data2['score'] * pf_data2['niche_grade'] 
pf_data2['sqft_li_to_sqft_lo_score'] = pf_data2['score'] * pf_data2['sqft_li_to_sqft_lo'] 
pf_data2['score^2'] = pf_data2['score'] * pf_data2['score'] 
pf_data2['school_grade^2'] = pf_data2['school_grade'] * pf_data2['school_grade'] 
pf_data2['school_grade_niche_grade'] = pf_data2['school_grade'] * pf_data2['niche_grade'] 
pf_data2['school_grade_score'] = pf_data2['school_grade'] * pf_data2['score']  
pf_data2['school_grade_sqft_li_to_sqft_lo'] = pf_data2['school_grade'] * pf_data2['sqft_li_to_sqft_lo'] 

In [28]:
all_feats = pf_data2.copy()

In [29]:
#Creates interaction between sqft_living and basement binary value

all_feats['s_b_b'] = all_feats['basement'] * all_feats['sqft_living'] 

#Creates interaction between waterfront and sqft_living

all_feats['w_b_s'] = all_feats['waterfront'] * all_feats['sqft_living']

In [30]:
after_transform_df = all_feats.copy()

In [37]:
af_list = ['years_since_updated', 's_b_b','w_b_s', 'bedrooms',
           'bathrooms','sqft_living','sqft_lot', 'floors','waterfront','view', 'distance_to_gas_works',
           'bath_to_bed','golden_ratio','sqft_li_to_sqft_lo','score','year_updated','basement','large_family',    
           'mansion', 'small_family','starter', 'high_end','high_mid','low','low_mid',
           'sqft_li_to_sqft_lo_score','score^2',
                       98001.0,                 98002.0,
                       98003.0,                 98004.0,
                       98005.0,                 98006.0,
                       98007.0,                 98008.0,
                       98010.0,                 98011.0,
                       98014.0,                 98019.0,
                       98022.0,                 98023.0,
                       98024.0,                 98027.0,
                       98028.0,                 98029.0,
                       98030.0,                 98031.0,
                       98032.0,                 98033.0,
                       98034.0,                 98038.0,
                       98039.0,                 98040.0,
                       98042.0,                 98045.0,
                       98052.0,                 98053.0,
                       98055.0,                 98056.0,
                       98058.0,                 98059.0,
                       98065.0,                 98070.0,
                       98072.0,                 98074.0,
                       98075.0,                 98077.0,
                       98092.0,                 98102.0,
                       98103.0,                 98105.0,
                       98106.0,                 98107.0,
                       98108.0,                 98109.0,
                       98112.0,                 98115.0,
                       98116.0,                 98117.0,
                       98118.0,                 98119.0,
                       98122.0,                 98125.0,
                       98126.0,                 98133.0,
                       98136.0,                 98144.0,
                       98146.0,                 98148.0,
                       98155.0,                 98166.0,
                       98168.0,                 98177.0,
                       98178.0,                 98188.0,
                       98198.0,                 98199.0,]  



In [38]:
after_transform_df = after_transform_df[af_list]

In [39]:
after_transform_df.shape

(4323, 97)

In [40]:
transformed_holdout = final_scaler.transform(after_transform_df)

In [41]:
final_answers = final_model.predict(transformed_holdout)

In [43]:
final_answers[:3]

array([317475.54017901, 210923.54017901, 116611.54017901])

In [44]:
df = pd.DataFrame(final_answers)
df.to_csv('housing_predictions_Jiji.csv')